# Machine Learning Fairness Algorithms Evaluation

#### Fairness in Machine Learning
The aim of a fairness algorithm is to avoid the outcome decisions are being made unfairly to certain groups or individuals.
- Algorithms are made by humans and trained by data which may be biased
- However, there are many definitions of fairness that cannot be optimized at the same time


#### Learning fair representations (LFR)
The main idea: map each individual, represented as a data point in a given input space, to a probability distribution in a new representation space. The aim of this new representation is to lose any information that can identify whether the person belongs to the protected subgroup, while retaining as much other information as possible.

A discriminative clustering model, where the prototypes act as the clusters

#### Prejudice Remover Regularizer (PR)

ADD DETAILS


### Data Pre-processing

In [101]:
import csv
import pandas as pd

import numpy as np
from scipy.special import softmax
import scipy.optimize as optim
from sklearn.preprocessing import StandardScaler

In [102]:
data = 'https://raw.githubusercontent.com/juliamblake1/ADS-Project-4/main/data/compas-scores-two-years_processed.csv'

In [103]:
compas_scores = pd.read_csv(data,header =0)
compas_scores.head()

,sex,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,...,score_text,v_decile_score,v_score_text,priors_count.1,start,end,event,custody_duration,jail_duration_hours,two_year_recid
0,Male,Greater than 45,Other,0,1,0,0,0,-1.0,1.0,...,Low,1,Low,0,0,327,0,7.0,23.627222,0
1,Male,25 - 45,African-American,0,3,0,0,0,-1.0,1.0,...,Low,1,Low,0,9,159,1,10.0,241.857222,1
2,Male,Less than 25,African-American,0,4,0,1,4,-1.0,1.0,...,Low,3,Low,4,0,63,0,0.0,26.058333,1
3,Male,25 - 45,Other,0,1,0,0,0,0.0,0.0,...,Low,1,Low,0,1,853,0,1.0,31.643889,0
4,Male,25 - 45,Caucasian,0,6,0,0,14,-1.0,1.0,...,Medium,2,Low,14,5,40,1,18.0,151.168333,1


In [104]:
# Filter the compas_scores DataFrame to include only rows where the 'race' column is either 'Caucasian' or 'African-American'.
# This creates a new DataFrame, compas_scores_race, containing data for these two specific racial groups.
compas_scores_race = compas_scores.loc[compas_scores['race'].isin(['Caucasian' ,'African-American'])]
compas_scores_race  # Display the resulting DataFrame.


,sex,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,...,score_text,v_decile_score,v_score_text,priors_count.1,start,end,event,custody_duration,jail_duration_hours,two_year_recid
1,Male,25 - 45,African-American,0,3,0,0,0,-1.0,1.0,...,Low,1,Low,0,9,159,1,10.0,241.857222,1
2,Male,Less than 25,African-American,0,4,0,1,4,-1.0,1.0,...,Low,3,Low,4,0,63,0,0.0,26.058333,1
4,Male,25 - 45,Caucasian,0,6,0,0,14,-1.0,1.0,...,Medium,2,Low,14,5,40,1,18.0,151.168333,1
6,Female,25 - 45,Caucasian,0,1,0,0,0,-1.0,1.0,...,Low,1,Low,0,2,747,0,3.0,70.886667,0
7,Male,Less than 25,Caucasian,0,3,0,0,1,428.0,308.0,...,Low,5,Medium,1,0,428,1,1.0,23.719444,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6893,Male,25 - 45,African-American,0,2,0,0,0,-1.0,1.0,...,Low,2,Low,0,0,529,1,0.0,22.444167,1
6894,Male,Less than 25,African-American,0,9,0,0,0,-1.0,1.0,...,High,9,High,0,0,169,0,20.0,20.930833,0
6895,Male,Less than 25,African-American,0,7,0,0,0,-1.0,1.0,...,Medium,5,Medium,0,1,860,0,2.0,45.681389,0
6896,Male,Less than 25,African-American,0,3,0,0,0,-1.0,1.0,...,Low,5,Medium,0,1,790,0,2.0,44.832778,0


In [105]:
# Convert the 'race' column into binary data using one-hot encoding. This results in a new DataFrame 'df_one' with binary columns for each race.
df_one = pd.get_dummies(compas_scores_race["race"])

# print(df_one)

# Concatenate the one-hot encoded DataFrame 'df_one' with the original 'compas_scores_race' DataFrame.
# This concatenation is done along the columns (axis=1).
df_two = pd.concat((df_one, compas_scores_race), axis=1)

# Remove the original 'race' column and the 'Caucasian' column as it is now redundant after one-hot encoding.
df_two = df_two.drop(["race", "Caucasian"], axis=1)

# Rename the 'African-American' column to 'race' for clarity. The resulting DataFrame 'new_df' now represents the race as a binary variable.
new_df = df_two.rename(columns={"African-American": "race"})
new_df


,race,sex,age_cat,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,...,score_text,v_decile_score,v_score_text,priors_count.1,start,end,event,custody_duration,jail_duration_hours,two_year_recid
1,1,Male,25 - 45,0,3,0,0,0,-1.0,1.0,...,Low,1,Low,0,9,159,1,10.0,241.857222,1
2,1,Male,Less than 25,0,4,0,1,4,-1.0,1.0,...,Low,3,Low,4,0,63,0,0.0,26.058333,1
4,0,Male,25 - 45,0,6,0,0,14,-1.0,1.0,...,Medium,2,Low,14,5,40,1,18.0,151.168333,1
6,0,Female,25 - 45,0,1,0,0,0,-1.0,1.0,...,Low,1,Low,0,2,747,0,3.0,70.886667,0
7,0,Male,Less than 25,0,3,0,0,1,428.0,308.0,...,Low,5,Medium,1,0,428,1,1.0,23.719444,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6893,1,Male,25 - 45,0,2,0,0,0,-1.0,1.0,...,Low,2,Low,0,0,529,1,0.0,22.444167,1
6894,1,Male,Less than 25,0,9,0,0,0,-1.0,1.0,...,High,9,High,0,0,169,0,20.0,20.930833,0
6895,1,Male,Less than 25,0,7,0,0,0,-1.0,1.0,...,Medium,5,Medium,0,1,860,0,2.0,45.681389,0
6896,1,Male,Less than 25,0,3,0,0,0,-1.0,1.0,...,Low,5,Medium,0,1,790,0,2.0,44.832778,0


In [106]:
new_df.two_year_recid.value_counts(), new_df.race.value_counts()

(0    3083
 1    2825
 Name: two_year_recid, dtype: int64,
 1    3534
 0    2374
 Name: race, dtype: int64)

## LFR Implementation
First the loss functions

In [107]:
np.random.seed(509)


def loss_x(x_new, x_initial):
    return np.mean(np.sum(np.square((x_new - x_initial))))


def loss_y(y_true, y_predicted):
    # logarithm is undefined in 0 which means y cant be 0 or 1 => we clip it
    y_true = np.clip(y_true, 1e-6, 0.999)
    y_predicted = np.clip(y_predicted, 1e-6, 0.999)

    log_loss = np.sum(y_true * np.log(y_predicted) +
                      (1. - y_true) * np.log(1. - y_predicted)) / len(y_true)

    return -log_loss


def loss_z(M_k_sensitive, M_k_non_sensitive):
    return np.sum(np.abs(M_k_sensitive - M_k_non_sensitive))

In [108]:
def distances(X, v, alpha):
    num_examples = X.shape[0]
    num_prototypes = v.shape[0]
    dists = np.zeros(shape=(num_examples, num_prototypes))

    # X = X.values  # converting to NumPy, this is needed in case you pass dataframe
    for i in range(num_examples):
        dist = np.square(X[i] - v)  # squarred distance
        dist_alpha = np.multiply(dist, alpha)  # multiplying by weights
        sum_ = np.sum(dist_alpha, axis=1)
        dists[i] = sum_

    return dists

In [109]:
def M_nk(dists):
    return softmax(-dists, axis=1)  # specifying axis is important


def M_k(M_nk):
    return np.mean(M_nk, axis=0)

In [110]:
def x_n_hat(M_nk, v):
    return M_nk @ v


def y_hat(M_nk, w):
    return M_nk @ w

Now for the optimization

In [111]:
def optim_objective(params, data_sensitive, data_non_sensitive, y_sensitive,
                    y_non_sensitive,  inference=False, NUM_PROTOTYPES=10, A_x=0.01, A_y=0.1, A_z=0.5,
                    print_every=100):
    optim_objective.iters += 1

    num_features = data_sensitive.shape[1]
    # extract values for each variable from params vector
    alpha_non_sensitive = params[:num_features]
    alpha_sensitive = params[num_features:2 * num_features]
    w = params[2 * num_features:2 * num_features + NUM_PROTOTYPES]
    v = params[2 * num_features + NUM_PROTOTYPES:].reshape(NUM_PROTOTYPES, num_features)

    dists_sensitive = distances(data_sensitive, v, alpha_sensitive)
    dists_non_sensitive = distances(data_non_sensitive, v, alpha_non_sensitive)

    # get probabilities of mappings
    M_nk_sensitive = M_nk(dists_sensitive)
    M_nk_non_sensitive = M_nk(dists_non_sensitive)

    # M_k only used for calcilating loss_y(statistical parity)
    M_k_sensitive = M_k(M_nk_sensitive)
    M_k_non_sensitive = M_k(M_nk_non_sensitive)
    L_z = loss_z(M_k_sensitive, M_k_non_sensitive)  # stat parity

    # get new representation of data
    x_hat_sensitive = x_n_hat(M_nk_sensitive, v)
    x_hat_non_sensitive = x_n_hat(M_nk_non_sensitive, v)
    # calculates how close new representation is to original data
    L_x_sensitive = loss_x(data_sensitive, x_hat_sensitive)
    L_x_non_sensitive = loss_x(data_non_sensitive, x_hat_non_sensitive)

    # get new values for labels
    y_hat_sensitive = y_hat(M_nk_sensitive, w)
    y_hat_non_sensitive = y_hat(M_nk_non_sensitive, w)
    # ensure how good new predictions are(log_loss)
    L_y_sensitive = loss_y(y_sensitive, y_hat_sensitive)
    L_y_non_sensitive = loss_y(y_non_sensitive, y_hat_non_sensitive)

    L_x = L_x_sensitive + L_x_non_sensitive
    L_y = L_y_sensitive + L_y_non_sensitive

    loss = A_x * L_x + A_y * L_y + A_z * L_z

    if optim_objective.iters % print_every == 0:
        print(f'loss on iteration {optim_objective.iters} : {loss}, L_x - {L_x * A_x} L_y - {L_y * A_y} L_z - {L_z * A_z}')

    if not inference:
        return loss
    if inference:
        return x_hat_sensitive, x_hat_non_sensitive, y_hat_sensitive, y_hat_non_sensitive

optim_objective.iters = 0

Now lets actually implement after we have all of the functions necessary

We are going to use a subset of the data for trial:

In [112]:
# Obtain a list of unique categories in the 'age_cat' column of the new_df DataFrame.
age_list = new_df['age_cat'].value_counts().index.to_list()

# Iterate through each age category in the list.
for section_name in age_list:
    # Create a new column for each age category in new_df.
    # This column will have binary values: 1 if the 'age_cat' matches the section_name, 0 otherwise.
    # This is achieved using the apply method with a lambda function.
    new_df[f'age_{section_name}'] = new_df['age_cat'].apply(lambda x: int(x == section_name))


In [113]:
use_df = new_df[['two_year_recid', 'race', 'priors_count', 'v_decile_score', 'age_25 - 45', 'age_Less than 25', 'age_Greater than 45']]
use_df

,two_year_recid,race,priors_count,v_decile_score,age_25 - 45,age_Less than 25,age_Greater than 45
1,1,1,0,1,1,0,0
2,1,1,4,3,0,1,0
4,1,0,14,2,1,0,0
6,0,0,0,1,1,0,0
7,1,0,1,5,0,1,0
...,...,...,...,...,...,...,...
6893,1,1,0,2,1,0,0
6894,0,1,0,9,0,1,0
6895,0,1,0,5,0,1,0
6896,0,1,0,5,0,1,0


In [114]:
# Define a variable to represent the threshold for determining race sensitivity.
RACE_SENSITIVE = 0

# Separate the data into two groups: sensitive and non-sensitive based on the race value.
# Sensitive data includes rows where the 'race' value is greater than RACE_SENSITIVE.
# Non-sensitive data includes rows where the 'race' value is less than or equal to RACE_SENSITIVE.
data_sensitive = use_df[use_df.race > RACE_SENSITIVE]
data_non_sensitive = use_df[use_df.race <= RACE_SENSITIVE]

# Extract the 'two_year_recid' column from both the sensitive and non-sensitive datasets
# to use as labels for each group.
y_sensitive = data_sensitive.two_year_recid
y_non_sensitive = data_non_sensitive.two_year_recid


In [115]:
print (f'Dataset contains {use_df.shape[0]} examples and {use_df.shape[1]} features')
print (f'From which {data_sensitive.shape[0]} belong to sensitive group and {data_non_sensitive.shape[0]} to non sensitive group ')

Dataset contains 5908 examples and 7 features
From which 3534 belong to sensitive group and 2374 to non sensitive group 


In [116]:
# Remove the 'two_year_recid' and 'race' columns from the sensitive and non-sensitive data.
# This is done as these columns are not needed for further processing or model training.
data_sensitive = data_sensitive.drop(['two_year_recid', 'race'], axis=1)
data_non_sensitive = data_non_sensitive.drop(['two_year_recid', 'race'], axis=1)

# Standardize (scale) the features in the sensitive and non-sensitive data.
# StandardScaler is used to transform the data such that its distribution will have a mean value of 0 and standard deviation of 1.
data_sensitive_scaled = StandardScaler().fit_transform(data_sensitive)
data_non_sensitive_scaled = StandardScaler().fit_transform(data_non_sensitive)


In [117]:
NUM_PROTOTYPES = 10
num_features = data_sensitive.shape[1]

params = np.random.uniform(size=(num_features * 2 + NUM_PROTOTYPES + NUM_PROTOTYPES * num_features))
# here we generate random weight for each of the features both for sensitive data
# and for non sensitive, hence num_features*2(in paper this is denoted as alpha)
# alphas are used for calculating distances


In [118]:
# Then NUM_PROTOTYPES is a weight for each prototype, this is multiplied with
# M_nk s and used for calculating y_hat

# Next is NUM_PROTOTYPES * num_features which is v(in paper), this is also used
# for calculating distances


bnd = [] # This is needed for l-bfgs algorithm
for i, _ in enumerate(params):
    if i < num_features * 2 or i >= num_features * 2 + NUM_PROTOTYPES:
        bnd.append((None, None))
    else:
        bnd.append((0, 1))

In [119]:
# Convert the data_sensitive and data_non_sensitive dataframes to NumPy arrays if they are not already.
# This conversion is necessary for compatibility with many machine learning algorithms and optimization functions.
data_sensitive_array = data_sensitive.to_numpy() if isinstance(data_sensitive, pd.DataFrame) else data_sensitive
data_non_sensitive_array = data_non_sensitive.to_numpy() if isinstance(data_non_sensitive, pd.DataFrame) else data_non_sensitive

# Convert the y_sensitive and y_non_sensitive series to NumPy arrays if they are not already.
# This ensures that the target variables are in the correct format for modeling.
y_sensitive_array = y_sensitive.to_numpy() if isinstance(y_sensitive, pd.Series) else y_sensitive
y_non_sensitive_array = y_non_sensitive.to_numpy() if isinstance(y_non_sensitive, pd.Series) else y_non_sensitive

# Optimize the parameters of the model using the L-BFGS-B algorithm.
# The optim_objective function is passed along with the initial parameters 'params' and the data arrays.
# The bounds for the optimization, the approximation gradient, maximum number of function evaluations (maxfun), and maximum number of iterations (maxiter) are also specified.
new_params = optim.fmin_l_bfgs_b(optim_objective, x0=params, epsilon=1e-5,
                                  args=(data_sensitive_array, data_non_sensitive_array,
                                        y_sensitive_array, y_non_sensitive_array),
                                  bounds=bnd, approx_grad=True, maxfun=1000,
                                  maxiter=1000)[0]

# Retrieve the new dataset representations and predicted labels (probabilities) using the optimized parameters.
# The 'inference' flag set to True indicates that the function should return the transformed datasets and predictions.
x_hat_sensitive, x_hat_nons, y_hat_sens, y_hat_nons = optim_objective(new_params, data_sensitive_array, data_non_sensitive_array,
                                        y_sensitive_array, y_non_sensitive_array, inference=True)


loss on iteration 100 : 2594.238495367817, L_x - 2593.8597389909964 L_y - 0.14732612203790585 L_z - 0.2314302547823241
loss on iteration 200 : 1511.5329118617365, L_x - 1511.1388457914225 L_y - 0.15086177982710702 L_z - 0.2432042904867579
loss on iteration 300 : 699.7749918080822, L_x - 699.3332029415662 L_y - 0.1773262864069457 L_z - 0.26446258010902446
loss on iteration 400 : 1411.2396170160837, L_x - 1410.640617311437 L_y - 0.14930133568747594 L_z - 0.4496983689594148
loss on iteration 500 : 4472.1088590217705, L_x - 4471.017041603485 L_y - 0.17366969252090306 L_z - 0.918147725764153
loss on iteration 600 : 593.2093126185594, L_x - 592.763188748729 L_y - 0.16892015096750065 L_z - 0.27720371886282014
loss on iteration 700 : 610.1143740978366, L_x - 609.7836055990108 L_y - 0.1546912741667692 L_z - 0.1760772246590766
loss on iteration 800 : 532.7235927862257, L_x - 532.3913572782825 L_y - 0.15222973430249487 L_z - 0.1800057736406682
loss on iteration 900 : 8200.743661582028, L_x - 8199

# Evaluation and Comparison

In [120]:
# This block of code is used for making binary predictions based on the probability thresholds.
# If the predicted probability (y_hat) is greater than or equal to 0.5, the event is predicted to occur (classified as 1),
# otherwise, it is predicted not to occur (classified as 0).

# Initialize empty lists to store predictions for sensitive and non-sensitive data.
y_preds_sens = []
y_preds_nons = []

# Iterate over each predicted probability in y_hat_sens (for sensitive data) and classify based on the threshold.
for i in range(y_hat_sens.shape[0]):
    if y_hat_sens[i] >= 0.5:
        y_preds_sens.append(1)  # Append 1 to the list if probability is greater than or equal to 0.5
    else:
        y_preds_sens.append(0)  # Append 0 otherwise

# Repeat the same classification process for y_hat_nons (for non-sensitive data).
for i in range(y_hat_nons.shape[0]):
    if y_hat_nons[i] >= 0.5:
        y_preds_nons.append(1)
    else:
        y_preds_nons.append(0)


In [121]:
#make a list of true labels for sensitive and nonsensitive data
y_sensitive = list(y_sensitive)
y_nonsensitive = list(y_non_sensitive)

In [122]:
# This block of code calculates the accuracy for sensitive and non-sensitive data groups
# and then computes the difference in accuracy between these two groups.

# Initialize counters for correct predictions in sensitive and non-sensitive data.
sens_correct = 0
nons_correct = 0

# Calculate the number of correct predictions for sensitive data.
for i in range(len(y_preds_sens)):
    if y_preds_sens[i] == y_sensitive[i]:
        sens_correct += 1  # Increment the counter if the prediction matches the actual value.

# Calculate the number of correct predictions for non-sensitive data.
for i in range(len(y_preds_nons)):
    if y_preds_nons[i] == y_nonsensitive[i]:
        nons_correct += 1

# Calculate the accuracy for sensitive and non-sensitive data groups.
sens_acc = sens_correct / len(y_preds_sens)  # Accuracy for sensitive data.
nons_acc = nons_correct / len(y_preds_nons)  # Accuracy for non-sensitive data.

# Calculate the difference in accuracy between the two groups.
diff = nons_acc - sens_acc

# Print the accuracy results and the difference in accuracy.
print('Sensitive Data Accuracy: ', sens_acc * 100)  # Convert to percentage.
print('Non-sensitive Data Accuracy: ', nons_acc * 100)  # Convert to percentage.
print('Accuracy Difference', diff * 100)  # Convert to percentage and display the accuracy difference.


Sensitive Data Accuracy:  59.62082625919638
Non-sensitive Data Accuracy:  63.352990732940185
Accuracy Difference 3.7321644737438042


In [123]:
import numpy as np

# Create an array of threshold values ranging from 0 to 1 (inclusive) with 101 evenly spaced values.
thresholds = np.linspace(0, 1, 101)

# Initialize variables to keep track of the best gap in accuracy between sensitive and non-sensitive groups
# and the corresponding thresholds that achieve this best gap.
best_gap = float('inf')  # Start with the largest possible gap.
best_thresh_sens = 0.5  # Initial threshold for sensitive group.
best_thresh_nons = 0.5  # Initial threshold for non-sensitive group.

# Iterate through all possible threshold combinations for sensitive and non-sensitive groups.
for thresh_sens in thresholds:
    # Generate predictions for sensitive data based on the current threshold.
    y_preds_sens = [1 if prob >= thresh_sens else 0 for prob in y_hat_sens]
    # Calculate accuracy for the sensitive group.
    sens_acc = sum(pred == true for pred, true in zip(y_preds_sens, y_sensitive)) / len(y_preds_sens)

    for thresh_nons in thresholds:
        # Generate predictions for non-sensitive data based on the current threshold.
        y_preds_nons = [1 if prob >= thresh_nons else 0 for prob in y_hat_nons]
        # Calculate accuracy for the non-sensitive group.
        nons_acc = sum(pred == true for pred, true in zip(y_preds_nons, y_nonsensitive)) / len(y_preds_nons)

        # Calculate the gap in accuracy between the two groups.
        gap = abs(sens_acc - nons_acc)

        # If the current gap is the smallest so far, update the best thresholds and the best gap.
        if gap < best_gap:
            best_gap = gap
            best_thresh_sens = thresh_sens
            best_thresh_nons = thresh_nons

# Print the best thresholds for each group and the smallest accuracy gap achieved.
print("Best Threshold for Sensitive Group:", best_thresh_sens)
print("Best Threshold for Non-sensitive Group:", best_thresh_nons)
print("Accuracy Gap:", best_gap)


Best Threshold for Sensitive Group: 0.51
Best Threshold for Non-sensitive Group: 0.47000000000000003
Accuracy Gap: 1.0250644956300015e-05
